In [ ]:
from collections import defaultdict
import numpy as np
import random
from matplotlib import pyplot as plt
import pandas as pd
import plotly.express as px

<h1> Fish Simulation </h1>

In [ ]:
def length_from_weight(weight):
    return (weight**(1/3.0)) / 2.36068 * random.gauss(1.0, 0.05)

class Fish:
    
    def __init__(self, weight_mean, weight_std, speed_factor_mean, speed_factor_std, 
                 min_depth, max_depth, max_y_coordinate=3.0):
        self.weight = max(random.gauss(weight_mean, weight_std), 0.1)
        self.length = length_from_weight(self.weight)
        self.height = 0.3 * self.length
        self.depth = random.uniform(min_depth, max_depth)
        self.speed = self.length * random.gauss(speed_factor_mean, speed_factor_std)
        self.is_sampled = False
        self.position = [-10, self.depth, random.uniform(-max_y_coordinate, max_y_coordinate)]
        
    def update_position(self, delta_t):
        delta_x = self.speed * delta_t
        self.position[0] += delta_x
        
    def get_position(self):
        return self.position
        
        
class Camera:
    
    def __init__(self, position, fov_degrees, aspect_ratio=0.75):
        self.position = position
        self.fov = fov_degrees * np.pi / 180.0
        self.vfov = 2 * np.arctan(np.tan(self.fov / 2) * aspect_ratio)
        self.pixel_width = 1000
        self.pixel_height = int(self.pixel_width * aspect_ratio)
        self.focal_length_pixel = (self.pixel_width / 2) / np.tan(self.fov / 2)
        
    @staticmethod
    def gen_p_capture(depth, a=1.0, b=2.5, default_p=1.0):
        if depth < a:
            return default_p
        else:
            return max(default_p * (b - depth) / (b - a), 0)
        
    def contains(self, fish):
        fish_position = fish.get_position()
        fish_segment_at_depth = (fish_position[0] - fish.length / 2.0, fish_position[0] + fish.length / 2.0)
        field_size = 2 * fish_position[1] * np.tan(self.fov / 2.0)
        field_center = self.position[0]
        field_segment_at_depth = (field_center - field_size / 2.0, field_center + field_size / 2.0)
        inside_horizontal_field = (fish_segment_at_depth[0] > field_segment_at_depth[0]) and \
            (fish_segment_at_depth[1] < field_segment_at_depth[1])
        
        vertical_fish_segment_at_depth = (fish_position[2] - fish.height / 2.0, fish_position[2] + fish.height / 2.0)
        vertical_field_segment_at_depth = (-fish_position[1] * np.tan(self.vfov / 2.0), fish_position[1] * np.tan(self.vfov / 2.0))
        inside_vertical_field = (vertical_fish_segment_at_depth[0] > vertical_field_segment_at_depth[0]) and \
            (vertical_fish_segment_at_depth[1] < vertical_field_segment_at_depth[1])
        
        if inside_horizontal_field and inside_vertical_field:
            return random.random() < self.gen_p_capture(fish_position[1])
        return False
        


In [ ]:
from matplotlib.colors import Normalize
from matplotlib import cm
from PIL import Image, ImageDraw

sm = cm.ScalarMappable(cmap=cm.get_cmap('Reds'), norm=Normalize(vmin=0.3, vmax=3.0))

def spawn_fish(fishes):
    fish = Fish(5.0, 0.8, 0.9, 0.05, 0.3, 3.0)
    fishes.append(fish)
    
    
def move_fish(t, t_new, fishes):
    delta_t = t_new - t
    for fish in fishes:
        fish.update_position(delta_t)
        
    fishes = [fish for fish in fishes if fish.get_position()[0] < 10.0]
    return fishes
    

def check_if_fully_visible(fish, left_camera, right_camera):
    return left_camera.contains(fish) and right_camera.contains(fish)
    
    
def trigger_capture(fishes, sampled_fishes, left_camera, right_camera, remove_dups=True):
    for fish in fishes:
        is_visible = check_if_fully_visible(fish, left_camera, right_camera)
        if is_visible:
            fish.is_sampled = True
            sampled_fishes.append(fish)
            
    if remove_dups:
        fishes = [fish for fish in fishes if fish.is_sampled == False]
    return fishes
            
    

def get_pixel_bbox(fish, camera):
    x_pixel = fish.position[0] * camera.focal_length_pixel / fish.position[1] + camera.pixel_width / 2.0
    y_pixel = -(fish.position[2] * camera.focal_length_pixel / fish.position[1]) + camera.pixel_height / 2.0
    length_pixel = fish.length * camera.focal_length_pixel / fish.position[1]
    height_pixel = fish.height * camera.focal_length_pixel / fish.position[1]
    bbox = [x_pixel-length_pixel/2.0, y_pixel-height_pixel/2.0, x_pixel+length_pixel/2.0, y_pixel+height_pixel/2.0]
    return [int(x) for x in bbox]
    
    
def draw_frame(fishes, left_camera, right_camera):
    im = Image.new('RGB', (left_camera.pixel_width, left_camera.pixel_height))
    draw = ImageDraw.Draw(im)
    for fish in reversed(sorted(fishes, key=lambda x: x.depth)):
        bbox = get_pixel_bbox(fish, left_camera)
        color = sm.to_rgba(fish.depth, bytes=True)
        draw.ellipse(tuple(bbox), fill=color[:3])
    return np.array(im)
        
    
    


In [ ]:
def generate_samples(FOV, FPS, aspect_ratio=0.75):
    fishes = []
    sampled_fishes = []
    left_camera = Camera((0, 0, 0), FOV, aspect_ratio=aspect_ratio)
    right_camera = Camera((0.105, 0, 0), FOV, aspect_ratio=aspect_ratio)
    print(left_camera.vfov)

    capture_times = list(np.arange(0, 100000, 1.0 / FPS))
    fish_spawn_times = list(np.cumsum(np.random.exponential(0.5, int(100000 * 0.18))))

#     im_arrs = []
    t = 0
    while len(capture_times) > 0 and len(fish_spawn_times) > 0:
        event_type = np.argmin([capture_times[0], fish_spawn_times[0]])
        if event_type == 0:
            t_new = capture_times[0]
            fishes = move_fish(t, t_new, fishes)
#             if 1000 < t_new < 1100:
#                 im_arr = draw_frame(fishes, left_camera, right_camera)
#                 im_arrs.append(im_arr)
            fishes = trigger_capture(fishes, sampled_fishes, left_camera, right_camera, remove_dups=True)
            t = t_new
            del capture_times[0]
        elif event_type == 1:
            t_new = fish_spawn_times[0]
            fishes = move_fish(t, t_new, fishes)
            spawn_fish(fishes)
            t = t_new
            del fish_spawn_times[0]

        if len(capture_times) % 100000 == 0:
            print(len(capture_times))

    return sampled_fishes


<h1> 4 kg </h1>

In [ ]:
fishes = generate_samples(54, 0.6)

In [ ]:
plt.hist([f.depth for f in fishes])

In [ ]:
fishes_2 = generate_samples(80, 8.0)

In [ ]:
plt.hist([f.depth for f in fishes_2])

In [ ]:
working_distances = [0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2]
depth_of_field = 0.4
s1s, s2s = [], []

for working_distance in working_distances:
    lo, hi = working_distance - 0.5*depth_of_field, working_distance + 0.5*depth_of_field
    s1 = len([f for f in fishes if f.depth > lo and f.depth < hi])
    s2 = len([f for f in fishes_2 if f.depth > lo and f.depth < hi])
    
    s1s.append(s1)
    s2s.append(s2)

In [ ]:
[y / x if x > 0 else None for x, y in zip(s1s, s2s)]

<h1> 5 kg </h1>

In [ ]:
fishes = generate_samples(54, 0.6)

In [ ]:
fishes_2 = generate_samples(80, 8.0)

In [ ]:
working_distances = [0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2]
depth_of_field = 0.4
s1s, s2s = [], []

for working_distance in working_distances:
    lo, hi = working_distance - 0.5*depth_of_field, working_distance + 0.5*depth_of_field
    s1 = len([f for f in fishes if f.depth > lo and f.depth < hi])
    s2 = len([f for f in fishes_2 if f.depth > lo and f.depth < hi])
    
    s1s.append(s1)
    s2s.append(s2)

In [ ]:
[y / x for x, y in zip(s1s, s2s)]

In [ ]:
plt.hist([f.depth for f in fishes])

In [ ]:
fs = []
for idx, im_arr in enumerate(im_arrs):
    im = Image.fromarray(im_arr)
    f = '/root/data/alok/biomass_estimation/playground/fov_simulation_6/im_{}.jpg'.format(idx)
    im.save(f)
    fs.append(f)

In [ ]:
import cv2

def stitch_frames_into_video(image_fs, video_f):
    im = cv2.imread(image_fs[0])
    height, width, layers = im.shape
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    video = cv2.VideoWriter(video_f, fourcc, 4, (width, height), True)
    for idx, image_f in enumerate(image_fs):
        if idx % 1000 == 0:
            print(idx)
        im = cv2.imread(image_f, cv2.IMREAD_COLOR)
        video.write(im)
    cv2.destroyAllWindows()
    video.release()

In [ ]:
stitch_frames_into_video(image_fs, '/data/alok/biomass_estimation/playground/fov_simulation.avi')

In [ ]:
[f.replace('/root', '') for f in fs]

In [ ]:
def generate_analysis_df(fov, fps, aspect_ratio=0.75):
    sampled_fishes = generate_samples(fov, fps, aspect_ratio=aspect_ratio)
    depth_cutoffs = list(np.arange(0.5, 2.5, 0.1))
    depth_buckets, sample_sizes, pct_errors = [], [], []
    for low_d, high_d in zip(depth_cutoffs, depth_cutoffs[1:]):
        depth_bucket = '{}-{}'.format(round(low_d, 1), round(high_d, 1))
        fish_subset = [fish for fish in sampled_fishes if low_d < fish.depth < high_d]
        sample_size = len(fish_subset)
        avg_weight = np.mean([fish.weight for fish in fish_subset])
        pct_error = (avg_weight - 8) / 8

        depth_buckets.append(depth_bucket)
        sample_sizes.append(sample_size)
        pct_errors.append(pct_error)

    analysis_df = pd.DataFrame({
        'depth_bucket': depth_buckets,
        'sample_size': sample_sizes,
        'pct_error': pct_errors
    })
    return analysis_df



In [ ]:
for fov in fov_list:
    for fps in fps_list:
        analysis_df = analysis_dfs[fov][fps]
        analysis_df.pct_error = ((analysis_df.pct_error * 8 + 8) - 4) / 4

In [ ]:
analysis_dfs = defaultdict(dict)
fov_list = [65, 70, 75, 80, 85, 90]
fps_list = [1.0, 2.0, 4.0, 8.0]
for fov in fov_list:
    for fps in fps_list:
        print('FOV: {}, FPS: {}'.format(fov, fps))
        analysis_df = generate_analysis_df(fov, fps)
        analysis_dfs[fov][fps] = analysis_df

In [ ]:
analysis_df = generate_analysis_df(54, 0.6)
fig, ax = plt.subplots(figsize=(20, 10))
ax.bar(analysis_df.depth_bucket, analysis_df.sample_size, label='sample size', color='red')
for tick in ax.get_xticklabels():
    tick.set_rotation(90)
ax.set_xlabel('Distance-from-camera bucket (m)')

ax2=ax.twinx()
ax2.plot(analysis_df.depth_bucket, analysis_df.pct_error, label='pct error', color='blue')
ax.grid()
ax.legend(loc='lower right')
ax2.legend(loc='upper right')
ax2.axhspan(-0.02, 0.02, color='red', alpha=0.3)

ax.set_ylabel('Sample Size')
ax2.set_ylabel('Pct. error')
plt.show()

In [ ]:
analysis_df = generate_analysis_df(70, 8)
fig, ax = plt.subplots(figsize=(20, 10))
ax.bar(analysis_df.depth_bucket, analysis_df.sample_size, label='sample size', color='red')
for tick in ax.get_xticklabels():
    tick.set_rotation(90)
ax.set_xlabel('Distance-from-camera bucket (m)')

ax2=ax.twinx()
ax2.plot(analysis_df.depth_bucket, analysis_df.pct_error, label='pct error', color='blue')
ax.grid()
ax.legend(loc='lower right')
ax2.legend(loc='upper right')
ax2.axhspan(-0.02, 0.02, color='red', alpha=0.3)

ax.set_ylabel('Sample Size')
ax2.set_ylabel('Pct. error')
plt.show()

In [ ]:
analysis_df = generate_analysis_df(90, 8, aspect_ratio=0.5)
fig, ax = plt.subplots(figsize=(20, 10))
ax.bar(analysis_df.depth_bucket, analysis_df.sample_size, label='sample size', color='red')
for tick in ax.get_xticklabels():
    tick.set_rotation(90)
ax.set_xlabel('Distance-from-camera bucket (m)')

ax2=ax.twinx()
ax2.plot(analysis_df.depth_bucket, analysis_df.pct_error, label='pct error', color='blue')
ax.grid()
ax.legend(loc='lower right')
ax2.legend(loc='upper right')
ax2.axhspan(-0.02, 0.02, color='red', alpha=0.3)


ax.set_ylabel('Sample Size')
ax2.set_ylabel('Pct. error')
plt.show()

In [ ]:
analysis_df = generate_analysis_df(90, 8, aspect_ratio=0.75)
fig, ax = plt.subplots(figsize=(20, 10))
ax.bar(analysis_df.depth_bucket, analysis_df.sample_size, label='sample size', color='red')
for tick in ax.get_xticklabels():
    tick.set_rotation(90)
ax.set_xlabel('Distance-from-camera bucket (m)')

ax2=ax.twinx()
ax2.plot(analysis_df.depth_bucket, analysis_df.pct_error, label='pct error', color='blue')
ax.grid()
ax.legend(loc='lower right')
ax2.legend(loc='upper right')
ax2.axhspan(-0.02, 0.02, color='red', alpha=0.3)


ax.set_ylabel('Sample Size')
ax2.set_ylabel('Pct. error')
plt.show()

In [ ]:
analysis_df = generate_analysis_df(90, 8, aspect_ratio=0.8)
fig, ax = plt.subplots(figsize=(20, 10))
ax.bar(analysis_df.depth_bucket, analysis_df.sample_size, label='sample size', color='red')
for tick in ax.get_xticklabels():
    tick.set_rotation(90)
ax.set_xlabel('Distance-from-camera bucket (m)')

ax2=ax.twinx()
ax2.plot(analysis_df.depth_bucket, analysis_df.pct_error, label='pct error', color='blue')
ax.grid()
ax.legend(loc='lower right')
ax2.legend(loc='upper right')
ax2.axhspan(-0.02, 0.02, color='red', alpha=0.3)

ax.set_ylabel('Sample Size')
ax2.set_ylabel('Pct. error')

plt.show()

In [ ]:
analysis_df = generate_analysis_df(90, 8, aspect_ratio=0.9)
fig, ax = plt.subplots(figsize=(20, 10))
ax.bar(analysis_df.depth_bucket, analysis_df.sample_size, label='sample size', color='red')
for tick in ax.get_xticklabels():
    tick.set_rotation(90)
ax.set_xlabel('Distance-from-camera bucket (m)')

ax2=ax.twinx()
ax2.plot(analysis_df.depth_bucket, analysis_df.pct_error, label='pct error', color='blue')
ax.grid()
ax.legend(loc='lower right')
ax2.legend(loc='upper right')
ax2.axhspan(-0.02, 0.02, color='red', alpha=0.3)

ax.set_ylabel('Sample Size')
ax2.set_ylabel('Pct. error')

plt.show()

In [ ]:
analysis_df = generate_analysis_df(90, 8, aspect_ratio=1.0)
fig, ax = plt.subplots(figsize=(20, 10))
ax.bar(analysis_df.depth_bucket, analysis_df.sample_size, label='sample size', color='red')
for tick in ax.get_xticklabels():
    tick.set_rotation(90)
ax.set_xlabel('Distance-from-camera bucket (m)')

ax2=ax.twinx()
ax2.plot(analysis_df.depth_bucket, analysis_df.pct_error, label='pct error', color='blue')
ax.grid()
ax.legend(loc='lower right')
ax2.legend(loc='upper right')
ax2.axhspan(-0.02, 0.02, color='red', alpha=0.3)

ax.set_ylabel('Sample Size')
ax2.set_ylabel('Pct. error')

plt.show()

In [ ]:
fig, axes = plt.subplots(len(fov_list), len(fps_list), figsize=(30, 20))
for i, fov in enumerate(fov_list):
    for j, fps in enumerate(fps_list):
        ax = axes[i, j]
        
        analysis_df = analysis_dfs[fov][fps]
        ax.bar(analysis_df.depth_bucket, analysis_df.sample_size, label='sample size', color='red')
        for tick in ax.get_xticklabels():
            tick.set_rotation(90)

        ax2 = ax.twinx()
        ax2.plot(analysis_df.depth_bucket, analysis_df.pct_error, label='pct error', color='blue')
        ax.grid()
        ax.legend()
        ax2.legend()
#         ax2.axhline(-0.02, color='red', linestyle='--')
        ax2.axhspan(-0.02, 0.02, color='red', alpha=0.3)
        
        ax.set_title('FOV: {}, FPS: {}'.format(fov, fps))

fig.subplots_adjust(hspace=1.0)


In [ ]:
fig, axes = plt.subplots(len(fov_list), len(fps_list), figsize=(30, 20))
for i, fov in enumerate(fov_list):
    for j, fps in enumerate(fps_list):
        ax = axes[i, j]
        
        analysis_df = analysis_dfs[fov][fps]
        ax.bar(analysis_df.depth_bucket, analysis_df.sample_size, label='sample size', color='red')
        for tick in ax.get_xticklabels():
            tick.set_rotation(90)

        ax2 = ax.twinx()
        ax2.plot(analysis_df.depth_bucket, analysis_df.pct_error, label='pct error', color='blue')
        ax.grid()
        ax.legend()
        ax2.legend()
#         ax2.axhline(-0.02, color='red', linestyle='--')
        ax2.axhspan(-0.02, 0.02, color='red', alpha=0.3)
        
        ax.set_title('FOV: {}, FPS: {}'.format(fov, fps))

fig.subplots_adjust(hspace=1.0)


In [ ]:
sampled_fishes = generate_samples(54, 0.6)

In [ ]:
depths = [fish.depth for fish in sampled_fishes]
weights = [fish.weight for fish in sampled_fishes]
plt.scatter(depths, weights)
plt.grid()
plt.show()

In [ ]:
sampled_fishes = generate_samples(90, 8.0)

In [ ]:
depths = [fish.depth for fish in sampled_fishes]
weights = [fish.weight for fish in sampled_fishes]
plt.scatter(depths, weights)
plt.grid()
plt.show()

In [ ]:
sampled_fishes = generate_samples(70, 8.0)

In [ ]:
depths = [fish.depth for fish in sampled_fishes]
weights = [fish.weight for fish in sampled_fishes]
plt.scatter(depths, weights)
plt.grid()
plt.show()

In [ ]:
!python3 -m pip install sympy

In [ ]:
from sympy.solvers import solve
from sympy import Symbol

In [ ]:
x = Symbol('x')
y = Symbol('y')
sols = solve([(x)**2 + (y)**2-1, (x-1.3)**2 + (y+1)**2-1], (x, y))

In [ ]:
for sol in sols:
    print(sol[0].is_real and sol[1].is_real)